In [ ]:
import pandas as pd
import numpy as np
import time
import calendar
from random import randint
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
import warnings
warnings.filterwarnings("ignore")
from geopy.geocoders import Nominatim #https://geopy.readthedocs.io/en/stable/

In [2]:
def data_old(df):
    
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

def data_analytics(df):
    
    df_null = (pd.DataFrame(df.isnull().sum(),
            columns = ['quantities number null'])
            .rename_axis('Columns')
            .reset_index())
    #df_null.reset_index(level=0, inplace=True)
    
    df_missing = (pd.DataFrame((df.isnull().sum() / len(df))*100,
            columns = ['percentage data missing'])
            .rename_axis('Columns')
            .reset_index())
    
    df_type = (pd.DataFrame(df.dtypes,
            columns = ['data type'])
            .rename_axis('Columns')
            .reset_index())
        
    df_describe = (pd.DataFrame(df.describe(include="all").transpose()
            .rename_axis('Columns'))
            .reset_index())

    df_1 = pd.merge(df_null, df_missing)
    df_2 = pd.merge(df_1, df_type)
    df   = pd.merge(df_2, df_describe)

    if df.shape[1] == 15:
        var_lst = [df["Columns"],df["quantities number null"],df["percentage data missing"],df["data type"].map(str),
                df["count"],df['unique'],df['top'],df['freq'],df['mean'],df['std'],
                df["min"],df["25%"],df["50%"],df["75%"],df["max"]]

        table = go.Table(header = dict(values = df.columns.tolist(),
                               font=dict(color='white'),
                               line = dict(color = ['#506784']),
                               fill = dict(color = ['#119DFF']),
                              ),
                 cells  = dict(values = var_lst,
                               line = dict(color = ['#506784']),
                               fill = dict(color = ["lightgrey",'#F5F8FF']),
                               format = [None] + [", .2f"] + [", .2f"] + [None] + [None] + [None] + [None] + [None] + [',.4f'] ,
                              ),
                 columnwidth = [155,100,100,100,100,100,100,100,100,100,100,100,100,100,100])
                
        layout = go.Layout(dict(title = "Table Analytics", title_x=0.5))
        figure = go.Figure(data=[table],layout=layout)
        box = py.iplot(figure)
    else:
        colorscale = [[0, '#4d004c'],[.5, '#f2e5ff'],[1, '#ffffff']]
        fig = ff.create_table(df, height_constant=25)
        fig.layout.width = 4700
        box = fig.show()
    
    return box

def data_new(df):
    
    if 'store_and_fwd_flag' in df.columns and 'id' in df.columns: 
        df = df.drop(columns=['store_and_fwd_flag'])
        df = df.drop(columns=['id'])
        
    if 'vendor_id' in df.columns:
        df['vendor_id'] = (df['vendor_id']
                           .map({1:'class_one',
                                 2:'class_two'}))
        df['vendor_id'] = df['vendor_id'].astype('category')
        
    if 'pickup_datetime' in df.columns and 'dropoff_datetime' in df.columns:
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
    
    if 'pickup_longitude' in df.columns and 'pickup_latitude' in df.columns:
        #df['pickup_longitude'] = df['pickup_longitude'].astype(str, errors = 'raise')
        #df['pickup_latitude'] = df['pickup_latitude'].astype(str, errors = 'raise')
        df["pickup_lat_lon"] = df['pickup_latitude'].map(str) + "," + df['pickup_longitude'].map(str)
        #df = df.drop(columns=['pickup_longitude', 'pickup_latitude'])
        
    if 'dropoff_longitude' in df.columns and 'dropoff_latitude' in df.columns:
        #df['dropoff_longitude'] = df['dropoff_longitude'].astype(str, errors = 'raise')
        #df['dropoff_latitude'] = df['dropoff_latitude'].astype(str, errors = 'raise')
        df["dropoff_lat_lon"] = df['dropoff_latitude'].map(str) + "," + df['dropoff_longitude'].map(str)
        #df = df.drop(columns=['dropoff_longitude', 'dropoff_latitude'])
    
    if 'trip_duration' in df.columns:
        df['trip_duration'] = df['dropoff_datetime'] - df['pickup_datetime']
        
    if 'dropoff_datetime' in df.columns and 'pickup_datetime' in df.columns:
        df['pickup_date'] = pd.to_datetime(df['pickup_datetime'].dt.date)
        df['pickup_day'] = df['pickup_datetime'].apply(lambda x : x.day)
        df['pickup_hour'] = df['pickup_datetime'].apply(lambda x : x.hour)
        df['pickup_day_of_week'] = df['pickup_datetime'].apply(lambda x : calendar.day_name[x.weekday()])
        df['dropoff_date'] = pd.to_datetime(df['dropoff_datetime'].dt.date)
        df['dropoff_day'] = df['dropoff_datetime'].apply(lambda x : x.day)
        df['dropoff_hour'] = df['dropoff_datetime'].apply(lambda x : x.hour)
        df['dropoff_day_of_week'] = df['dropoff_datetime'].apply(lambda x : calendar.day_name[x.weekday()])
        df["month"] = pd.DatetimeIndex(df["pickup_datetime"]).month
        df["month_name"] = df["month"].map({1:"JAN",2:"FEB",3:"MAR",
                                            4:"APR",5:"MAY",6:"JUN",
                                            7:"JUL",8:"AUG",9:"SEP",
                                            10:"OCT",11:"NOV",12:"DEC"
                                            })
        
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

def distance_KM(df):
    """
    A fórmula de Haversine determina a distância do grande círculo entre dois pontos em uma esfera, dadas suas longitudes e latitudes.
        Formula:
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = (sin(dlat/2))^2 + cos(lat1) cos(lat2) (sin(dlon/2))^2
            c = 2 * atan2( sqrt(a), sqrt(1-a) )
            d = R * c (where R is the radius of the Earth)
    """
    R = 6373.0

    pickup_lat  = np.radians(df["pickup_latitude"])
    pickup_lon  = np.radians(df["pickup_longitude"])
    dropoff_lat = np.radians(df["dropoff_latitude"])
    dropoff_lon = np.radians(df["dropoff_longitude"])

    dist_lon = dropoff_lon - pickup_lon
    dist_lat = dropoff_lat - pickup_lat

    #Formula
    a = (np.sin(dist_lat/2))**2 + np.cos(pickup_lat) * np.cos(dropoff_lat) * (np.sin(dist_lon/2))**2 
    c = 2 * np.arctan2( np.sqrt(a), np.sqrt(1-a) ) 
    d = R * c

    df["trip_distance_km"] = round(d,3)
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

def data_address(df):
    df = df.sample(n=10)
    
    geolocator = Nominatim(user_agent="myGeocoder")
    
    if 'pickup_lat_lon' in df.columns and 'dropoff_lat_lon' in df.columns:
        df['pickup_address'] = df.apply(lambda row: geolocator.reverse(row['pickup_lat_lon']).raw,axis=1)
        df['dropoff_address'] = df.apply(lambda row: geolocator.reverse(row['dropoff_lat_lon']).raw,axis=1)
    
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

def dictionary(df):
    
    data = (df
           .reset_index(drop=True)['pickup_address']
           .to_dict())
    
    data = data[randint(0,9)]
    
    for chave in data.keys():
        print(f'Chave = {chave} e Valor = {data[chave]}')
        
def final_step_with_api(df):
    
    if 'pickup_lat_lon' in df.columns and 'dropoff_lat_lon' in df.columns:
        df['pickup_country'] = (df['pickup_address']
                                .apply(lambda row: row['address']['country']))
        df['dropoff_country'] = (df['dropoff_address']
                                .apply(lambda row: row['address']['country']))
        df['pickup_state'] = (df['pickup_address']
                                .apply(lambda row: row['address']['state']))
        df['dropoff_state'] = (df['dropoff_address']
                                .apply(lambda row: row['address']['state']))
        
    df.drop(columns=['pickup_address','dropoff_address'], axis=1, inplace=True)
    
    df = df[['vendor_id', 'pickup_datetime', 'dropoff_datetime', 
             'passenger_count', 'trip_duration', 'pickup_state', 
             'pickup_country', 'dropoff_state', 'dropoff_country',
             'pickup_date', 'month', 'month_name', 'pickup_day', 
             'pickup_hour', 'pickup_day_of_week', 'dropoff_date', 
             'dropoff_day', 'dropoff_hour', 'dropoff_day_of_week',
             'trip_distance_km',
             'pickup_latitude', 'pickup_longitude','pickup_lat_lon', 
             'dropoff_latitude', 'dropoff_longitude', 'dropoff_lat_lon']]
        
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df

def final_step(df):

    df = df[['vendor_id', 'pickup_datetime', 'dropoff_datetime', 
             'passenger_count', 'trip_duration', 'pickup_date', 
             'month', 'month_name', 'pickup_day', 'pickup_hour', 
             'pickup_day_of_week', 'dropoff_date', 'dropoff_day', 
             'dropoff_hour', 'dropoff_day_of_week', 'trip_distance_km',
             'pickup_latitude', 'pickup_longitude','pickup_lat_lon', 
             'dropoff_latitude', 'dropoff_longitude', 'dropoff_lat_lon']]
    
    df.to_csv('new_base_taxi.csv')
        
    print(f"O dataset contém {df.shape[0]} amostras e "f"{df.shape[1]} colunas")
    return df